target is to maximize New to brand view attributed sold items 

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns

# Load the Excel file to explore its content and structure
campaign_data = pd.read_csv('Amazon Ads All.csv')

# Display the first few rows of the dataframe and the column names to understand the dataset
campaign_data.head(), campaign_data.columns

(   ADV_ID    AD_ID         SKU     placement_slot  impressions  clicks  \
 0   84373  6370624  9985955750  offsite_mobileweb         1089       0   
 1   84373  1295011  7875390725    offsite_desktop         2043       0   
 2   84373  6370624  9985955750  offsite_mobileweb         1452       0   
 3   84373  1295011  7875390725    offsite_desktop         1312       0   
 4   84373  1295011  7875390725  offsite_mobileapp            8       0   
 
    auction_cost  adjusted_cost hit_day_utc  \
 0      10.89000        8.79000    02/05/24   
 1      26.61464       19.44791    01/05/24   
 2      14.52000       11.68400    01/05/24   
 3      16.59940       11.94286    30/04/24   
 4       0.11608        0.07161    01/05/24   
 
                                        targeting_secondary    vertical  \
 0             category=Automotive Replacement Control Arms  Automotive   
 1               category=Automotive Replacement Brake Kits  Automotive   
 2             category=Automotive Repl

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np


# Attempt to parse dates in the '%d/%m/%y' format
campaign_data['hit_day_utc_parsed'] = pd.to_datetime(campaign_data['hit_day_utc'], format='%d/%m/%y', errors='coerce')

# Parse remaining unparsed dates in the '%Y-%m-%d %H:%M:%S' format
mask = campaign_data['hit_day_utc_parsed'].isna()
campaign_data.loc[mask, 'hit_day_utc_parsed'] = pd.to_datetime(campaign_data.loc[mask, 'hit_day_utc'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Check for any remaining non-parsed dates
remaining_non_parsed = campaign_data[campaign_data['hit_day_utc_parsed'].isna()]
if not remaining_non_parsed.empty:
    print("Remaining non-parsed dates:")
    print(remaining_non_parsed)


# Extract month and day of the week from parsed dates
campaign_data['month_name'] = campaign_data['hit_day_utc_parsed'].dt.strftime('%B')  # Get month names
campaign_data['day_name'] = campaign_data['hit_day_utc_parsed'].dt.strftime('%A')    # Get day names

# Fill missing values
campaign_data.fillna({
    'vertical': 'unknown',
    'sub_vertical': 'unknown',
    'ntb_click_attributed_units_sold': 0,
    'ntb_view_attributed_units_sold': 0,
    'ntb_click_attributed_orders': 0,
    'ntb_view_attributed_orders': 0,
    'view_attributed_units_sold': 0,
    'view_attributed_orders': 0
}, inplace=True)

# Filter data for NTB reach
ntb_data = campaign_data[(campaign_data['ntb_view_attributed_units_sold'] > 0)]

# Define features and target for NTB reach
features_ntb = ['impressions', 'targeting_secondary', 
                'placement_slot', 'month_name', 'day_name']
target_ntb = 'ntb_view_attributed_units_sold'

# Prepare the data
X_ntb = ntb_data[features_ntb]
y_ntb = ntb_data[target_ntb]

# One-hot encode categorical features
preprocessor_ntb = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['targeting_secondary', 'placement_slot', 'month_name', 'day_name'])
    ],
    remainder='passthrough'
)

# Define and train the Gradient Boosting Regressor for NTB reach
model_ntb = Pipeline(steps=[
    ('preprocessor', preprocessor_ntb),
    ('regressor', GradientBoostingRegressor(random_state=42))
])

X_train_ntb, X_test_ntb, y_train_ntb, y_test_ntb = train_test_split(X_ntb, y_ntb, test_size=0.2, random_state=42)
model_ntb.fit(X_train_ntb, y_train_ntb)

# Predict and evaluate the NTB reach model
y_pred_ntb = model_ntb.predict(X_test_ntb)
mse_ntb = mean_squared_error(y_test_ntb, y_pred_ntb)
mae_ntb = mean_absolute_error(y_test_ntb, y_pred_ntb)
r2_ntb = r2_score(y_test_ntb, y_pred_ntb)

# Display the evaluation metrics for NTB reach model
print(f"NTB Reach Model - Mean Squared Error: {mse_ntb}")
print(f"NTB Reach Model - Mean Absolute Error: {mae_ntb}")
print(f"NTB Reach Model - R-squared: {r2_ntb}")

# Get feature names after one-hot encoding
encoded_feature_names = model_ntb.named_steps['preprocessor'].transformers_[0][1].get_feature_names_out(['targeting_secondary', 'placement_slot', 'month_name', 'day_name'])
all_feature_names = np.concatenate((encoded_feature_names, ['impressions']))

# Calculate feature importance
importance_ntb = model_ntb.named_steps['regressor'].feature_importances_
feature_importance_ntb = pd.DataFrame({
    'Feature': all_feature_names,
    'Importance': importance_ntb
}).sort_values(by='Importance', ascending=False)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
print("Feature Importance for NTB Reach Model:")
print(feature_importance_ntb)


NTB Reach Model - Mean Squared Error: 0.2831011715350623
NTB Reach Model - Mean Absolute Error: 0.2540523086877353
NTB Reach Model - R-squared: 0.10042924980404
Feature Importance for NTB Reach Model:
                                                                            Feature  \
143                          targeting_secondary_category=Canned & Packaged Chicken   
142                            targeting_secondary_category=Canned & Jarred Seafood   
642                  targeting_secondary_category=Probiotic Nutritional Supplements   
413                           targeting_secondary_category=Household Cleaning Tools   
925                                                                     impressions   
921                                                                 day_name_Sunday   
755                             targeting_secondary_category=Swimming Pool Chlorine   
861                              targeting_secondary_category=Women's Running Shoes   
915             

In [5]:
pd.set_option('display.max_colwidth', 50)
print("Feature Importance for NTB Reach Model:")
print(feature_importance_ntb)

Feature Importance for NTB Reach Model:
                                               Feature    Importance
143  targeting_secondary_category=Canned & Packaged...  1.360246e-01
142  targeting_secondary_category=Canned & Jarred S...  1.241055e-01
642  targeting_secondary_category=Probiotic Nutriti...  9.414953e-02
413  targeting_secondary_category=Household Cleanin...  8.992122e-02
925                                        impressions  5.286046e-02
921                                    day_name_Sunday  5.201158e-02
755  targeting_secondary_category=Swimming Pool Chl...  4.206884e-02
861  targeting_secondary_category=Women's Running S...  3.903546e-02
915                                month_name_February  3.657817e-02
453  targeting_secondary_category=Job Site & Securi...  2.863930e-02
923                                   day_name_Tuesday  2.708186e-02
916                                   month_name_March  2.487620e-02
922                                  day_name_Thursday  1.81546

Hyperparameter Tuning

In [6]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__learning_rate': [0.01, 0.05, 0.1],
    'regressor__max_depth': [3, 4, 5]
}

# Define and train the Gradient Boosting Regressor with GridSearchCV
model_ntb_tuned = Pipeline(steps=[
    ('preprocessor', preprocessor_ntb),
    ('regressor', GradientBoostingRegressor(random_state=42))
])

grid_search = GridSearchCV(model_ntb_tuned, param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(X_train_ntb, y_train_ntb)

# Best parameters and model evaluation
best_model_ntb = grid_search.best_estimator_
y_pred_ntb_tuned = best_model_ntb.predict(X_test_ntb)
mse_ntb_tuned = mean_squared_error(y_test_ntb, y_pred_ntb_tuned)
mae_ntb_tuned = mean_absolute_error(y_test_ntb, y_pred_ntb_tuned)
r2_ntb_tuned = r2_score(y_test_ntb, y_pred_ntb_tuned)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Tuned NTB Reach Model - Mean Squared Error: {mse_ntb_tuned}")
print(f"Tuned NTB Reach Model - Mean Absolute Error: {mae_ntb_tuned}")
print(f"Tuned NTB Reach Model - R-squared: {r2_ntb_tuned}")

# Get feature importance for the best model
importance_ntb_tuned = best_model_ntb.named_steps['regressor'].feature_importances_
feature_importance_ntb_tuned = pd.DataFrame({
    'Feature': all_feature_names,
    'Importance': importance_ntb_tuned
}).sort_values(by='Importance', ascending=False)

print("Feature Importance for Tuned NTB Reach Model:")
print(feature_importance_ntb_tuned)


Best parameters: {'regressor__learning_rate': 0.1, 'regressor__max_depth': 4, 'regressor__n_estimators': 300}
Tuned NTB Reach Model - Mean Squared Error: 0.26679485780845014
Tuned NTB Reach Model - Mean Absolute Error: 0.24095021413047105
Tuned NTB Reach Model - R-squared: 0.15224352804400998
Feature Importance for Tuned NTB Reach Model:
                                               Feature  Importance
925                                        impressions    0.115046
143  targeting_secondary_category=Canned & Packaged...    0.081882
142  targeting_secondary_category=Canned & Jarred S...    0.075410
921                                    day_name_Sunday    0.068943
642  targeting_secondary_category=Probiotic Nutriti...    0.057768
413  targeting_secondary_category=Household Cleanin...    0.054891
915                                month_name_February    0.042448
914                                   month_name_April    0.036521
916                                   month_name_March   

In [7]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation
cv_scores = cross_val_score(model_ntb, X_ntb, y_ntb, cv=5, scoring='r2')
print(f"Cross-Validation R-squared Scores: {cv_scores}")
print(f"Mean R-squared Score: {np.mean(cv_scores)}")


Cross-Validation R-squared Scores: [0.10975166 0.08748145 0.09027403 0.09185206 0.0956116 ]
Mean R-squared Score: 0.09499415908533357
